In [1]:
import nibabel as nib
import numpy as np
import os

### CHANGE THE BASE DIRECTORY WITH YOUR CORRECT PATH OF THE FOLDER THAT CONTAINS ALL RESAMPLED DATA

In [2]:
# Define the base directory containing patient folders
base_directory = r'D:\Studying\Fourth year\GP\Datasets\All Data Resampled'

In [7]:
# Loop over each subfolder in the base directory
for subfolder in os.listdir(base_directory):
    subfolder_path = os.path.join(base_directory, subfolder)

    # Check if the path is a directory
    if os.path.isdir(subfolder_path):
        # Loop over each patient folder in the subfolder
        for patient_folder in os.listdir(subfolder_path):
            patient_path = os.path.join(subfolder_path, patient_folder)

            # Check if the path is a directory (to avoid files)
            if os.path.isdir(patient_path):
                print(f'Processing patient: {patient_folder}')
                
                # Define the path for the images
                nifti_file = os.path.join(patient_path, 'Images', f'{patient_folder}.nii.gz')
                
                # Load the NIfTI image file
                img = nib.load(nifti_file)
                img_data = img.get_fdata()

                # Check if the "Contours" folder exists
                contour_exists = os.path.exists(os.path.join(patient_path, 'Contours'))
                
                # If the "Contours" folder exists, load the mask data
                if contour_exists:
                    masked_file = os.path.join(patient_path, 'Contours', f'{patient_folder}.nii.gz')
                    masked_img = nib.load(masked_file)
                    masked_data = masked_img.get_fdata()
                
                num_slices = img_data.shape[2]

                # Determine the number of channels for the output array
                num_channels = 2 if contour_exists else 1
                
                # Initialize an array to store combined slices
                combined_slices = np.zeros((img_data.shape[0], img_data.shape[1], num_slices, num_channels)) 

                # Loop through each slice and assign image and mask (if exists)
                for i in range(num_slices):
                    combined_slices[:, :, i, 0] = img_data[:, :, i]  # Assign the image slice
                    if contour_exists:
                        combined_slices[:, :, i, 1] = masked_data[:, :, i]  # Assign the mask slice

                # Directory to save the combined slices
                output_directory = os.path.join(patient_path, 'Slices')
                os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

                # Loop through each slice and save it
                for i in range(num_slices):
                    if contour_exists:
                        # Create a new array to hold both the image and contour
                        combined_slice = np.zeros((img_data.shape[0], img_data.shape[1], 2))  # Shape: (height, width, 2)
                        
                        # Assign the image and contour to the new array
                        combined_slice[:, :, 0] = combined_slices[:, :, i, 0]  # Image slice
                        combined_slice[:, :, 1] = combined_slices[:, :, i, 1]  # Contour slice
                        
                        # Determine the infarction value based on the dataset folder name
                        parent_folder = os.path.basename(subfolder_path)  # Get the name of the parent folder
                        if parent_folder.startswith('EMIDEC'):
                            infarction_value = 3
                        elif parent_folder.startswith('MyoPS'):
                            infarction_value = 1220
                        else:
                            infarction_value = None  # Handle unexpected cases if needed

                        # Check for infarction and no-reflow
                        if infarction_value is not None and np.any(combined_slice[:, :, 1] == infarction_value):
                            slice_filename = f'{patient_folder}_slice_{i+1}_Inf_NoReflowN.npy'
                            if np.any(combined_slice[:, :, 1] == 4): 
                                slice_filename = f'{patient_folder}_slice_{i+1}_Inf_NoReflowY.npy'
                        else:
                            slice_filename = f'{patient_folder}_slice_{i+1}_NoInf_NoReflowN.npy'
                            if np.any(combined_slice[:, :, 1] == 4):  
                                slice_filename = f'{patient_folder}_slice_{i+1}_NoInf_NoReflowY.npy'

                    else:
                        # Only include the image slice if no contour is available
                        combined_slice = np.zeros((img_data.shape[0], img_data.shape[1], 1))  # Shape: (height, width, 1)
                        combined_slice[:, :, 0] = combined_slices[:, :, i, 0]  # Image slice
                        slice_filename = f'{patient_folder}_slice_{i+1}.npy'

                    # Save the combined slice as .npy file
                    np.save(os.path.join(output_directory, slice_filename), combined_slice)

                print(f'Combined slices for {patient_folder} saved successfully.')


Processing patient: Case_101
Combined slices for Case_101 saved successfully.
Processing patient: Case_102
Combined slices for Case_102 saved successfully.
Processing patient: Case_103
Combined slices for Case_103 saved successfully.
Processing patient: Case_104
Combined slices for Case_104 saved successfully.
Processing patient: Case_105
Combined slices for Case_105 saved successfully.
Processing patient: Case_106
Combined slices for Case_106 saved successfully.
Processing patient: Case_107
Combined slices for Case_107 saved successfully.
Processing patient: Case_108
Combined slices for Case_108 saved successfully.
Processing patient: Case_109
Combined slices for Case_109 saved successfully.
Processing patient: Case_110
Combined slices for Case_110 saved successfully.
Processing patient: Case_111
Combined slices for Case_111 saved successfully.
Processing patient: Case_112
Combined slices for Case_112 saved successfully.
Processing patient: Case_113
Combined slices for Case_113 saved 